In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import joblib

# --------------------
# 1. Load Dataset
# --------------------
df = pd.read_csv('chronickidneydisease.csv')

# Example: Adjust column names if they have spaces
df.columns = df.columns.str.strip()

# Define target & features
target = 'class'  # Adjust if your target column is named differently
X = df.drop(columns=[target])
y = df[target]

# If target is categorical like 'ckd'/'notckd', encode to 0/1
y = y.replace({'ckd': 1, 'notckd': 0})

# Identify column types
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()

# --------------------
# 2. Preprocessing
# --------------------
# Numeric pipeline: handle missing, then scale
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Categorical pipeline: handle missing, then one-hot encode
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ]
)

# --------------------
# 3. Full Pipeline with Model
# --------------------
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# --------------------
# 4. Train-Test Split & Training
# --------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)

# --------------------
# 5. Evaluate
# --------------------
y_pred = pipeline.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# --------------------
# 6. Save Pipeline & Feature Names
# --------------------
save_data = {
    'pipeline': pipeline,
    'feature_names': X.columns.tolist()
}
joblib.dump(save_data, 'CKD.pkl')

print("✅ Model pipeline saved as CKD.pkl")